In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../../MorphoMolMonteCarlo/"))
    Pkg.activate("Project.toml")
end

using MorphoMol

if split(pwd(), "/")[end] == "MorphoMolMonteCarlo"
    cd(joinpath(@__DIR__, "../MMMCNotebooks/"))
    Pkg.activate("Project.toml")
end

using PyCall
using JLD2
using LinearAlgebra
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MMMC/MorphoMolMonteCarlo`
  Activating project at `~/Doktor/Code/MMMC/MMMCNotebooks`
Precompiling GLMakie
  ✓ StatsFuns → StatsFunsChainRulesCoreExt
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ KernelDensity
  ✓ MeshIO
  ✓ Makie
  ✓ GLMakie
  6 dependencies successfully precompiled in 79 seconds. 234 already precompiled.
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
Precompiling DistributionsTestExt
  ✓ Distributions → DistributionsTestExt
  1 dependency successfully precompiled in 6 seconds. 43 already precompiled.
[ Info: Precompiling DistributionsTestExt [ffbe0ea5-a612-5ff7-aaf5-cac02eef3019]
┌ Warning: Module Distributions with build ID fafbfcfd-a36f-e4df-0000-a087527d1bad is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile_

In [2]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_diagram(points, dim):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
    # no cohomology
    dualize = False
    # create VRU decomposition object, does not perform reduction yet
    dcmp = oin.Decomposition(fil, dualize)
    rp = oin.ReductionParams()
    rp.compute_u = rp.compute_v = True
    rp.n_threads = 1
    # perform reduction
    dcmp.reduce(rp)
    # now we can acess V, R and U
    # indices are sorted_ids of simplices == indices in fil.cells()
    V = dcmp.v_data
    simplices = fil.simplices()
    dgm_dim = dcmp.diagram(fil, include_inf_points=False)[dim]
    return dgm_dim
"""

function get_diagram(points::Vector{Vector{Float64}}, dim::Int = 1)
    py"get_diagram"(points, dim)
end

function get_total_persistence(dgm::Matrix{Float64})
    sum(dgm[:,2] - dgm[:,1])
end


get_total_persistence (generic function with 1 method)

In [3]:
function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [4]:
function get_diagrams(folder)
    files = readdir(folder)
    n = length(files)
    [get_diagram(txt_to_points("$(folder)$(i-1).txt")) for i in 1:n]
end

function get_tps(dgms)
    [get_total_persistence(dgm) for dgm in dgms]
end

function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

get_thetas (generic function with 1 method)

In [13]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    dgms = get_diagrams(folder)
    tps = get_tps(dgms)
    fsols = get_fsols(folder)
    thetas = get_thetas(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps fsols thetas
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [14]:
folder = "../../Data/measure_paths/txts/6r7m/"
for subdir in readdir(folder)
    if subdir != ".DS_Store"
        folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
    end
end

In [19]:
# dgms = get_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# tps = get_tps(dgms)
# fsols = get_fsols(folder)
# thetas = get_thetas(folder);

@load "../../Data/measure_paths/jld2s/6r7m/1.jld2" dgms tps fsols thetas
xs = [i for i in 1:length(dgms)];

In [22]:
f = Figure(fontsize = 7)
pd_ax = Axis(f[1, 1:3], title = "Persistence Diagram")
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(dgms), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(fsols[$x])))

dgm_points = @lift([Point2f(dgms[$x][i,1], dgms[$x][i,2]) for i in 1:size(dgms[$x])[1]])

ms = 5
scatter!(pd_ax, dgm_points, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, fsols, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)

In [30]:
@load "../../Data/jld2/single_subunits/6r7m_protor.jld2" template_mol template_radii x_init
ssu_diagram = get_diagram([Vector{Float64}(center) for center in eachcol(template_mol)]);

In [60]:
filtered = [filter(x -> !any([all(x .≈ p) for p in eachrow(ssu_diagram)]), eachrow(dgm)) for dgm in dgms];

679-element Vector{Vector{SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}}}:
 [[24.66531991487093, 24.711746116222134], [26.795241234337023, 26.79540851460957], [54.875972029422364, 55.12860833654298], [54.57486301020108, 55.17307442114338], [56.67309225037497, 56.69737918637567], [64.03405376238, 64.04327923322913], [66.72384925574988, 66.81354777645583], [63.66890160195087, 66.92459130891604], [80.71674107064825, 83.27409651263078], [84.11145385046747, 84.13902165529105], [84.3058375652494, 84.33359648685389]]
 [[15.15983186797925, 15.585481177938863], [17.842169198373988, 20.187995760516138], [20.287593734377854, 20.288114844099617], [18.120994937556215, 20.79726061802652], [26.008158084483114, 26.04369083095652], [30.035209744869405, 30.05329990004829], [43.73802772255015, 44.79545275844259], [43.63929690749005, 44.81276668363542], [56.263773812186805, 56.26493914065971], [57.382233806549536, 57.40498137372406], [55.83350581179568, 57.544501

In [66]:
v_filtered = [[Vector{Float64}(elem) for elem in dgm] for dgm in filtered];

In [73]:
v_filtered[1][1]

2-element Vector{Float64}:
 24.66531991487093
 24.711746116222134

In [70]:
v_filtered[1][1]

2-element Vector{Float64}:
 24.66531991487093
 24.711746116222134

In [74]:
f = Figure(fontsize = 7)
pd_ax = Axis(f[1, 1:3], title = "Persistence Diagram")
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(v_filtered), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(fsols[$x])))

dgm_points = @lift([Point2f(v_filtered[$x][i][1], v_filtered[$x][i][2]) for i in 1:size(v_filtered[$x])[1]])

ms = 5
scatter!(pd_ax, dgm_points, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, fsols, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)

GLMakie.Screen(...)